In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Tạo điểm thành phần cho Môn 1 và Môn 2 với 4 thành phần và trọng số khác nhau
def generate_scores(num_students, weights):
    scores = np.random.uniform(0, 10, (num_students, 4))  # Điểm ngẫu nhiên từ 0 đến 10
    total_scores = np.dot(scores, weights) / 100
    return scores, total_scores

# Trọng số cho Môn 1 và Môn 2
weights_mon1 = [10, 10, 20, 60]
weights_mon2 = [10, 20, 20, 50]

# Sinh dữ liệu
scores_mon1, total_scores_mon1 = generate_scores(500, weights_mon1)
scores_mon2, total_scores_mon2 = generate_scores(500, weights_mon2)

# Gộp dữ liệu của cả hai môn học
scores = np.vstack([scores_mon1, scores_mon2])
# print(scores)
total_scores = np.concatenate([total_scores_mon1, total_scores_mon2])

# Phân loại điểm theo thang điểm yêu cầu
def classify_grade(score):
    if score < 4:
        return 'F'
    elif 4 <= score < 6:
        return 'D'
    elif 6 <= score < 6.5:
        return 'C'
    elif 6.5 <= score < 7:
        return 'C+'
    elif 7 <= score < 8:
        return 'B'
    elif 8 <= score < 8.5:
        return 'B+'
    elif 8.5 <= score < 9:
        return 'A'
    else:
        return 'A+'

grades = np.array([classify_grade(score) for score in total_scores])
# print(grades)

# Lấy 3 cột đầu làm X và cột cuối làm y
scores = np.round(scores, 2)

X = scores[:, :3]
print(X.shape)
y = scores[:, -1]
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


(1000, 3)
(1000,)


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def create_dnn_model(input_shape):
    model = Sequential()
    model.add(Dense(100, input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1))  # Một neuron đầu ra cho dự đoán điểm số liên tục
    return model

# Tạo và biên dịch mô hình
model = create_dnn_model((X_train.shape[1],))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# Đánh giá mô hình
loss, mae = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test MAE: {mae}")


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 18.3925 - mae: 3.5075 - val_loss: 8.5718 - val_mae: 2.4686
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.9271 - mae: 2.6357 - val_loss: 8.2331 - val_mae: 2.4394
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.2051 - mae: 2.6840 - val_loss: 8.2847 - val_mae: 2.4473
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.8094 - mae: 2.6330 - val_loss: 8.2071 - val_mae: 2.4462
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.3902 - mae: 2.7073 - val_loss: 8.1102 - val_mae: 2.4340
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.1711 - mae: 2.7244 - val_loss: 7.9852 - val_mae: 2.4242
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.3154 - mae: 2.5572 - val_loss: 8.1524 - val_mae: 2.4386
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.8305 - mae: 2.6185 - val_loss: 7.8579 - val_mae: 2.4161
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.2925 - m

In [4]:
# Dự đoán điểm số
prediction = model.predict(np.array([[8,7,9]]))
print("Predictions: ", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Predictions:  [[4.7479973]]


In [5]:
# Lưu mô hình CNN dưới dạng tệp HDF5 (hoặc SavedModel)
model.save('cuoicungdiemthi.h5')

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Reshape dữ liệu để phù hợp với CNN (samples, features, 1)
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # (1000, 3, 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)  # (200, 3, 1)

# Xây dựng mô hình CNN
def create_cnn_model(input_shape):
    model = Sequential()
    
    # Lớp Conv1D đầu tiên với 64 bộ lọc và kích thước bộ lọc là 1 (thử giảm kích thước bộ lọc)
    model.add(Conv1D(64, 1, activation='relu', input_shape=input_shape))
    
    # Lớp MaxPooling1D để giảm kích thước
    model.add(MaxPooling1D(1))  # Thử dùng kích thước maxpooling = 1
    
    # Lớp Conv1D thứ hai với 128 bộ lọc và kích thước bộ lọc là 1
    model.add(Conv1D(128, 1, activation='relu'))
    
    # Lớp MaxPooling1D thứ hai để giảm kích thước
    model.add(MaxPooling1D(1))  # Thử dùng kích thước maxpooling = 1
    
    # Lớp Flatten để chuyển đổi dữ liệu từ 2D thành 1D
    model.add(Flatten())
    
    # Lớp Dense để thêm các lớp hoàn thiện, với 100 neuron và hàm kích hoạt ReLU
    model.add(Dense(100, activation='relu'))
    
    # Dropout để tránh overfitting
    model.add(Dropout(0.1))
    
    # Lớp Dense cuối cùng để dự đoán một giá trị liên tục (dùng activation='linear' cho dự đoán số thực)
    model.add(Dense(1))  # Dự đoán một giá trị liên tục
    
    return model

# Định nghĩa input_shape phù hợp với dữ liệu đầu vào (3 đặc trưng, 1 chiều cho mỗi đặc trưng)
input_shape = (3, 1)  # Dữ liệu có 3 đặc trưng

# Tạo mô hình CNN
model_cnn = create_cnn_model(input_shape)

# Biên dịch mô hình
model_cnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Huấn luyện mô hình
model_cnn.fit(X_train_cnn, y_train, epochs=30, batch_size=32, validation_data=(X_test_cnn, y_test))

# Đánh giá mô hình CNN
loss, mae = model_cnn.evaluate(X_test_cnn, y_test)
print(f"CNN Test loss: {loss}, Test MAE: {mae}")


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 19.5133 - mae: 3.7061 - val_loss: 8.5986 - val_mae: 2.4622
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.5488 - mae: 2.7293 - val_loss: 8.7594 - val_mae: 2.4918
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.8150 - mae: 2.7307 - val_loss: 9.0567 - val_mae: 2.5287
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.2281 - mae: 2.6797 - val_loss: 9.4858 - val_mae: 2.5814
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.5109 - mae: 2.7422 - val_loss: 8.7914 - val_mae: 2.5083
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.9632 - mae: 2.6763 - val_loss: 8.7130 - val_mae: 2.5010
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.2744 - mae: 2.5854 - val_loss: 7.8745 - val_mae: 2.4336
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.9425 - mae: 2.5590 - val_loss: 7.9884 - val_mae: 2.4402
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.0529 -

In [7]:
# Dự đoán điểm số
prediction = model_cnn.predict(np.array([[8,7,9]]))
print("Predictions: ", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Predictions:  [[5.243809]]


In [8]:
# Lưu mô hình CNN dưới dạng tệp HDF5 (hoặc SavedModel)
model_cnn.save('cuoicungdiemthi_cnn.h5')

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

# Reshape dữ liệu để phù hợp với RNN (samples, time_steps, features)
X_train_rnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # (1000, 3, 1)
X_test_rnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)  # (200, 3, 1)

# Xây dựng mô hình RNN với 7 lớp
def create_rnn_model(input_shape):
    model = Sequential()

    # Thêm 5 lớp SimpleRNN với 64 đơn vị và hàm kích hoạt 'relu'
    for _ in range(5):
        model.add(SimpleRNN(64, activation='relu', return_sequences=True, input_shape=input_shape))

    # Thêm 1 lớp Dropout để tránh overfitting
    model.add(Dropout(0.1))
    
    # Lớp Dense để dự đoán một giá trị liên tục
    model.add(Dense(1))  # Dự đoán một giá trị liên tục
    
    return model

# Định nghĩa input_shape cho RNN (3 đặc trưng)
input_shape = (3, 1)

# Tạo mô hình RNN
model_rnn = create_rnn_model(input_shape)

# Biên dịch mô hình
model_rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Huấn luyện mô hình RNN
model_rnn.fit(X_train_rnn, y_train, epochs=30, batch_size=32, validation_data=(X_test_rnn, y_test))

# Đánh giá mô hình RNN
loss, mae = model_rnn.evaluate(X_test_rnn, y_test)
print(f"RNN Test loss: {loss}, Test MAE: {mae}")


Epoch 1/30


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 25.1928 - mae: 4.1730 - val_loss: 10.3267 - val_mae: 2.6626
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 13.1987 - mae: 3.0134 - val_loss: 10.1988 - val_mae: 2.6644
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11.6222 - mae: 2.8378 - val_loss: 10.2346 - val_mae: 2.6703
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11.6251 - mae: 2.8469 - val_loss: 9.0660 - val_mae: 2.5439
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10.5398 - mae: 2.7385 - val_loss: 8.7214 - val_mae: 2.5115
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.7493 - mae: 2.6468 - val_loss: 7.9408 - val_mae: 2.4342
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.0966 - mae: 2.5846 - val_loss: 7.8299 - val_mae: 2.4291
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.9807 - mae: 2.5375 - val_loss: 7.8997 - val_mae: 2.4351
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.3791 - mae: 2.

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Reshape dữ liệu để phù hợp với LSTM (samples, time_steps, features)
X_train_lstm = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # (1000, 3, 1)
X_test_lstm = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)  # (200, 3, 1)

# Xây dựng mô hình LSTM với 7 lớp
def create_lstm_model(input_shape):
    model = Sequential()

    # Thêm 5 lớp LSTM với 64 đơn vị và hàm kích hoạt 'relu'
    for _ in range(5):
        model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=input_shape))
    
    # Thêm 1 lớp Dropout để tránh overfitting
    model.add(Dropout(0.1))
    
    # Lớp Dense để dự đoán một giá trị liên tục
    model.add(Dense(1))  # Dự đoán một giá trị liên tục
    
    return model

# Định nghĩa input_shape cho LSTM (3 đặc trưng)
input_shape = (3, 1)

# Tạo mô hình LSTM
model_lstm = create_lstm_model(input_shape)

# Biên dịch mô hình
model_lstm.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Huấn luyện mô hình LSTM
model_lstm.fit(X_train_lstm, y_train, epochs=30, batch_size=32, validation_data=(X_test_lstm, y_test))

# Đánh giá mô hình LSTM
loss, mae = model_lstm.evaluate(X_test_lstm, y_test)
print(f"LSTM Test loss: {loss}, Test MAE: {mae}")


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 35.1853 - mae: 5.2037 - val_loss: 26.1282 - val_mae: 4.3284
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 32.0903 - mae: 4.8487 - val_loss: 20.9504 - val_mae: 3.7646
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 25.6799 - mae: 4.2640 - val_loss: 17.3823 - val_mae: 3.3984
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 20.6428 - mae: 3.7460 - val_loss: 15.9292 - val_mae: 3.2353
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 18.9408 - mae: 3.5744 - val_loss: 12.5452 - val_mae: 2.8892
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 14.4765 - mae: 3.1332 - val_loss: 10.2103 - val_mae: 2.6498
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 12.4585 - mae: 2.9388 - val_loss: 8.5480 - val_mae: 2.4886
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 9.6756 - mae: 2.6366 - val_loss: 9.3011 - val_mae: 2.5721
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 

In [11]:
# Dự đoán điểm số
prediction = model_rnn.predict(np.array([[8,7,9]]))
print("Predictions: ", prediction[0][2][0])
# Dự đoán điểm số
prediction = model_lstm.predict(np.array([[8,7,9]]))
print("Predictions: ", prediction[0,2,0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step
Predictions:  4.187786
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step
Predictions:  4.8062453


In [12]:
# Lưu mô hình CNN dưới dạng tệp HDF5 (hoặc SavedModel)
model_rnn.save('cuoicungdiemthi_rnn.h5')


In [13]:
# Lưu mô hình CNN dưới dạng tệp HDF5 (hoặc SavedModel)
model_lstm.save('cuoicungdiemthi_ltsm.h5')
